In [ ]:
from simple_salesforce import Salesforce
import pandas as pd
from dotenv import load_dotenv
from datetime import date
from r2r_pipelines import export_db
import os

In [ ]:
import r2r_pipelines as r2r

In [ ]:
from dotenv import load_dotenv  
load_dotenv()

In [8]:
SF_USERNAME = 'insights@taylors.edu.my'
SF_PASSWORD = 'Xuy35125'
SF_SECURITY_TOKEN = 'GPzpmdXt6wjBeCBzIYZoaJi9'

In [9]:
# Connect to Salesforce
sf = Salesforce(username=SF_USERNAME, password=SF_PASSWORD, security_token=SF_SECURITY_TOKEN)

In [10]:
new_query = sf.query_all("""Select 
Id,
LeadId, 
IsDeleted, 
NewValue,
OldValue,
CreatedDate
from 
LeadHistory 
WHERE Field = 'Status'
AND CreatedDate > 2022-01-01T00:00:00.000Z""")

In [11]:
def flatten_sf_record(record, parent_key='', sep='_'):
    items = []
    for k, v in record.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if new_key in ['attributes_type', 'attributes_url']:
            continue  # skip these keys
        if isinstance(v, dict):
            items.extend(flatten_sf_record(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# Flatten all Salesforce records
flattened_records = [flatten_sf_record(rec) for rec in new_query['records']]


In [12]:
lead_history = pd.DataFrame(flattened_records)

In [13]:
lead_history

,Id,LeadId,IsDeleted,NewValue,OldValue,CreatedDate
0,0172j0000Hcj1NbAQI,00Q0I00000lMSW4UAO,False,Invalid,Lost,2023-03-29T00:10:46.000+0000
1,0172j0000Hcj1XlAQI,00Q0I00000lMSW4UAO,False,Invalid (3 Attempts),Invalid,2023-03-29T00:13:55.000+0000
2,0172j0000Hcj1bRAQQ,00Q0I00000lMSW4UAO,False,Invalid,Invalid (3 Attempts),2023-03-29T00:14:43.000+0000
3,0172j0000Hcj1XWAQY,00Q0I00000lMSWOUA4,False,Invalid (3 Attempts),Invalid,2023-03-29T00:13:55.000+0000
4,0172j0000Hcj1bCAQQ,00Q0I00000lMSWOUA4,False,Invalid,Invalid (3 Attempts),2023-03-29T00:14:43.000+0000
...,...,...,...,...,...,...
494617,017Mg00007UB4uWIAT,00QMg00000D9dfNMAR,False,Qualified Converted,Warm,2025-08-06T05:54:16.000+0000
494618,017Mg00007UCOfyIAH,00QMg00000D9j4rMAB,False,Hot,New,2025-08-06T06:10:06.000+0000
494619,017Mg00007UHUU1IAP,00QMg00000D9oFqMAJ,False,Warm,New,2025-08-06T06:50:43.000+0000
494620,017Mg00007UHYHcIAP,00QMg00000D9oFqMAJ,False,Qualified Converted,Warm,2025-08-06T06:53:56.000+0000


In [ ]:
from datetime import datetime

lead_history['CreatedDate'] = lead_history['CreatedDate'].apply(
    lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.000%z').date() if pd.notnull(x) else None
)


In [ ]:
second_query = sf.query_all("""Select 
Id,
CreatedDate, 
LeadSource, 
Lead_status__c, 
ConvertedAccountId, 
ConvertedOpportunityId, 
ConvertedDate, 
CYCLE__c,
Intake_Year__c                            
from 
Lead
where Intake_Year__c in ('2022','2023','2024','2025','2026') or CreatedDate > 2022-01-01T00:00:00.000Z""")

In [ ]:
def flatten_sf_record(record, parent_key='', sep='_'):
    items = []
    for k, v in record.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if new_key in ['attributes_type', 'attributes_url']:
            continue  # skip these keys
        if isinstance(v, dict):
            items.extend(flatten_sf_record(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# Flatten all Salesforce records
flattened_records = [flatten_sf_record(rec) for rec in second_query['records']]

In [ ]:
leads = pd.DataFrame(flattened_records)

In [ ]:
from datetime import datetime

leads['CreatedDate'] = leads['CreatedDate'].apply(
    lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.000%z').date() if pd.notnull(x) else None
)

In [ ]:
leads=leads.rename(columns={
    'Id':'lead_id',
    'CreatedDate':'validfrom',
    'Lead_status__c':'lead_status',
    'CYCLE__c':'cycle',
    'Intake_Year__c':'intake_year',
    'LeadSource':'lead_source',
    'ConvertedAccountId':'converted_account_id',
    'ConvertedOpportunityId':'converted_opportunity_id',
    'ConvertedDate':'converted_date'})

In [ ]:
lead_history=lead_history.rename(columns={
    'Id':'lead_history_id',
    'LeadId':'lead_id',
    'IsDeleted':'isdeleted',
    'NewValue':'new_value',
    'OldValue':'old_value',
    'CreatedDate':'validto'})

In [ ]:
#merged_df = pd.merge(filtered_df, filtered_df_2, left_on='leadid', right_on='lead_id', how='left')
merged_df = pd.merge(leads, lead_history, left_on='lead_id', right_on='lead_id', how='left')

In [ ]:
# --- 1. Date Conversion with Error Handling ---
merged_df['validfrom'] = pd.to_datetime(merged_df['validfrom'], format='%d/%m/%Y', errors='coerce')
merged_df['validto'] = pd.to_datetime(merged_df['validto'], format='%d/%m/%Y', errors='coerce')
merged_df['converted_date'] = pd.to_datetime(merged_df['converted_date'], format='%Y-%m-%d', errors='coerce')

# --- DIAGNOSTIC: Check for NaT values after conversion ---
print("--- NaT Value Check (after conversion) ---")
print(f"NaT values in 'validfrom': {merged_df['validfrom'].isna().sum()}")
print(f"NaT values in 'validto': {merged_df['validto'].isna().sum()}")
print(f"NaT values in 'converted_date': {merged_df['converted_date'].isna().sum()}")
print("-------------------------------------------\n")

# Filter out rows with invalid dates
merged_df.dropna(subset=['validfrom', 'validto'], inplace=True)
if merged_df.empty:
    print("Warning: merged_df is empty after dropping rows with invalid dates.")
    exit()

# --- 2. Construct the SCD Type 2 table records ---
df_history_sorted = merged_df.sort_values(by=['lead_id', 'validfrom', 'validto']).copy()
scd_rows = []

common_attrs = [
    'lead_source', 'converted_account_id', 'converted_opportunity_id',
    'cycle', 'converted_date', 'intake_year'
]

for lead_id, group in df_history_sorted.groupby('lead_id'):
    first_event = group.iloc[0]
    
    # Handle initial 'old_value' record
    initial_status = first_event['old_value']
    initial_validto = first_event['validto']
    
    # SPECIAL CASE: If status is "New", use validfrom as modifieddate
    if initial_status == 'New':
        initial_modifieddate = first_event['validfrom']
    else:
        initial_modifieddate = pd.NaT
    
    initial_row = {
        'lead_id': lead_id,
        'status': initial_status,
        'modifieddate': initial_modifieddate,
        'validto': initial_validto,
    }
    for attr in common_attrs:
        initial_row[attr] = first_event[attr]
    scd_rows.append(initial_row)
    
    last_validto = initial_validto

    # Process each 'new_value' event
    for _, record in group.iterrows():
        current_status = record['new_value']
        current_validto = record['validto']
        
        # SPECIAL CASE: If status is "New", use validfrom as modifieddate
        if current_status == 'New':
            current_modifieddate = record['validfrom']
        else:
            current_modifieddate = last_validto
        
        current_row = {
            'lead_id': lead_id,
            'status': current_status,
            'modifieddate': current_modifieddate,
            'validto': current_validto,
        }
        for attr in common_attrs:
            current_row[attr] = record[attr]
        scd_rows.append(current_row)
        
        last_validto = current_validto

# Create raw SCD DataFrame
scd_df_raw = pd.DataFrame(scd_rows)
scd_df_raw = scd_df_raw.sort_values(by=['lead_id', 'modifieddate']).reset_index(drop=True)

# --- 3. Collapse Contiguous Same-Status Periods ---
scd_df_raw['prev_status'] = scd_df_raw.groupby('lead_id')['status'].shift(1)
scd_df_raw['prev_validto'] = scd_df_raw.groupby('lead_id')['validto'].shift(1)

scd_df_raw['new_record_flag'] = (
    (scd_df_raw['status'] != scd_df_raw['prev_status']) |
    (scd_df_raw['modifieddate'] != scd_df_raw['prev_validto']) |
    (scd_df_raw['prev_status'].isna()) |
    (scd_df_raw['modifieddate'].isna())
)

scd_df_raw['group_id'] = scd_df_raw.groupby('lead_id')['new_record_flag'].cumsum()

final_scd_df = scd_df_raw.groupby(['lead_id', 'group_id']).agg(
    status=('status', 'first'),
    modifieddate=('modifieddate', lambda x: x.iloc[0] if pd.notna(x.iloc[0]) else pd.NaT),
    validto=('validto', 'max'),
    lead_source=('lead_source', 'first'),
    converted_account_id=('converted_account_id', 'first'),
    converted_opportunity_id=('converted_opportunity_id', 'first'),
    cycle=('cycle', 'first'),
    converted_date=('converted_date', 'first'),
    intake_year=('intake_year', 'first')
).reset_index()

final_scd_df.drop(columns=['group_id'], inplace=True)

# --- 4. Determine 'current_flag' ---
MAX_OBSERVED_END_DATE = merged_df['validto'].max()
max_validto_per_lead = final_scd_df.groupby('lead_id')['validto'].transform('max')

final_scd_df['current_flag'] = (
    (final_scd_df['validto'] == max_validto_per_lead) &
    (final_scd_df['validto'] == MAX_OBSERVED_END_DATE)
).astype(int)

# --- 5. Format final output ---
column_order = [
    'lead_id', 'status', 'modifieddate', 'validto', 'lead_source',
    'converted_account_id', 'converted_opportunity_id', 'cycle',
    'converted_date', 'intake_year', 'current_flag'
]

final_scd_df = final_scd_df[column_order]

# Format dates
# for col in ['modifieddate', 'validto', 'converted_date']:
#    if col in final_scd_df.columns and pd.api.types.is_datetime64_any_dtype(final_scd_df[col]):
#        final_scd_df[col] = final_scd_df[col].dt.strftime('%d/%m/%Y').replace('NaT', 'NaN')

for col in ['modifieddate', 'validto', 'converted_date']:
    if col in final_scd_df.columns:
        final_scd_df[col] = pd.to_datetime(final_scd_df[col], errors='coerce') \
                                .dt.strftime('%d/%m/%Y')

print("\n--- Final SCD Type 2 Lead Status History ---")
print(final_scd_df)

In [ ]:
final_scd_df

In [ ]:
final_scd_df.dtypes

In [ ]:
# Convert specific columns to datetime
columns_to_convert = ['modifieddate', 'validto', 'converted_date']

for col in columns_to_convert:
    final_scd_df[col] = pd.to_datetime(final_scd_df[col], errors='coerce')

In [ ]:
final_scd_df=final_scd_df.rename(columns={
    'status':'lead_status'})

In [ ]:
from sqlalchemy.types import Date

engine = export_db.marcommdb_connection()
final_scd_df.to_sql('leads_history_staging', engine, schema='staging', if_exists='replace', index=False,dtype={'modifieddate': Date, 'validto': Date, 'converted_date': Date})

### Leads

In [ ]:
import sys
sys.path.append('c:/Users/112363/dna_pipeline/dna_pipelines')  # Adjust this to your actual path

from r2r_pipelines import export_db

In [ ]:
from simple_salesforce import Salesforce
import pandas as pd
from dotenv import load_dotenv
from datetime import date
from r2r_pipelines import export_db
import os

In [ ]:
SF_USERNAME = 'insights@taylors.edu.my'
SF_PASSWORD = 'Xuy35125'
SF_SECURITY_TOKEN = 'GPzpmdXt6wjBeCBzIYZoaJi9'

# Connect to Salesforce
sf = Salesforce(username=SF_USERNAME, password=SF_PASSWORD, security_token=SF_SECURITY_TOKEN)

In [ ]:
leads_query = sf.query_all("""Select 
Id,
CreatedDate, 
LeadSource, 
Lead_status__c, 
ConvertedAccountId, 
ConvertedOpportunityId, 
ConvertedDate, 
CYCLE__c,
Intake_Year__c                            
from 
Lead
where Intake_Year__c in ('2022','2023','2024','2025','2026') or CreatedDate > 2022-01-01T00:00:00.000Z""")

In [ ]:
def flatten_sf_record(record, parent_key='', sep='_'):
    items = []
    for k, v in record.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if new_key in ['attributes_type', 'attributes_url']:
            continue  # skip these keys
        if isinstance(v, dict):
            items.extend(flatten_sf_record(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# Flatten all Salesforce records
flattened_records = [flatten_sf_record(rec) for rec in leads_query['records']]

In [ ]:
leads = pd.DataFrame(flattened_records)

In [ ]:
leads.rename(columns={'Id': 'lead_id','CreatedDate':'created_date','Lead_status__c':'lead_status',
                      'ConvertedAccountId':'converted_account_id','ConvertedOpportunityId':'converted_opportunity_id',
                      'ConvertedDate':'converted_date','CYCLE__c':'cycle','Intake_Year__c':'intake_year'}, inplace=True)

In [ ]:
leads

In [ ]:
from datetime import datetime

leads['created_date'] = leads['created_date'].apply(
    lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.000%z').date() if pd.notnull(x) else None
)

In [ ]:
from sqlalchemy.types import Date
engine = export_db.marcommdb_connection()

leads.to_sql(
    'leads_staging',
    engine,
    schema='staging',
    if_exists='replace',
    index=False,
    dtype={'created_date': Date,'converted_date': Date}
)